<a href="https://colab.research.google.com/github/Meii0606/Breaking-reCAPTCHAv2/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-414/714: Deep Learning Systems - Final Project


## **Sparse Matrix Multiplication on Graph Neural Network**

**By Andrew Zhang, Jinkai Qiu & Yimei Wu**


---

In this project, we are going to implement **sparse matrix** class supported in Needle, **forward and backward pass of sparse matrix multiplication**, and its **application on Graph Neural Network(GNN)**.

In this notebook, we are going to show how to **define sparse matrix**, perform **sparse matrix multiplication** and compare it with normal dense matrix multiplication. In addition, we will also show how it can be used in **GNN training.**



## 1. Clone Required Repo and Install Required Packages

In [3]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/10714
!mkdir -p final_proj
%cd /content/drive/MyDrive/10714/final_proj
!git clone https://github.com/AndrewZhang76/gnn_with_spmm.git
%cd gnn_with_spmm/
!git pull
!pip3 install pybind11

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/10714
/content/drive/MyDrive/10714/final_proj
fatal: destination path 'gnn_with_spmm' already exists and is not an empty directory.
/content/drive/MyDrive/10714/final_proj/gnn_with_spmm
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 159.41 KiB | 214.00 KiB/s, done.
From https://github.com/AndrewZhang76/gnn_with_spmm
   119ff14..dff09b2  main       -> origin/main
Updating ecf1830..dff09b2
error: Your local changes to the following files would be overwritten by merge:
	python/needle/ops/ops_mathematic.py
Please commit your changes or stash them before you merge.
Aborting


## 2. Build

In [4]:
%cd /content/drive/MyDrive/10714/final_proj/gnn_with_spmm/
!make

/content/drive/MyDrive/10714/final_proj/gnn_with_spmm
-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.13.6")
CMake Warning (dev) at CMakeLists.txt:57 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- CUDA_FOUND: TRUE
-- Found cuda, building cuda backend
-- Configuring done (0.7s)
-- Generating done (1.0s)
-- Build files have been written to: /content/drive/MyDrive/10714/final_proj/gnn_with_spmm/build
make[1]: Entering directory '/content/drive/MyDrive/10714/final_proj/gnn_with_spmm/build'
make[2]: Entering directory '/content/drive/MyDrive/10714/final_proj/gnn_with_spmm/build'
make[3]: Entering directory '/content/drive/MyDrive/10714/final_proj/gnn_with_spmm/build'
make[3]: Leaving directory '/content/drive/M

In [5]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


## 3. Sparse Matrix Definiation.
In this project, we defined a new way to represent a sparse matrix(a type of matrix that contains a significant number of zero elements compared to the total number of elements in the matrix.) - **COO (Coordinate) format**. \
The COO (Coordinate) format is a representation of a sparse matrix that stores only the nonzero elements along with their row and column indices. It is efficient in terms of memory usage for sparse matrices because it avoids storing zero values.\
### Key Components:


1.   Values (data): A list or array of the nonzero elements in the matrix.
2.   Row indices (row): A list or array specifying the row index for each nonzero element.
3. Column indices (col): A list or array specifying the column index for each nonzero element.

###Properties:
1. Flexible: Allows easy manipulation, such as matrix construction from nonzero entries.
2. Duplicates: COO format allows duplicate entries. To obtain the actual matrix, these duplicates need to be summed.
3. Conversion: Often converted to other formats like CSR (Compressed Sparse Row) or CSC (Compressed Sparse Column) for efficient matrix operations.






#### First, we are going to randomly generate a sparse matrix in normal NDArray format. It is a 10 * 10 matrix with 10 non-zero elements.

In [ ]:
%cd /content/drive/MyDrive/10714/final_proj/gnn_with_spmm/python/needle
import numpy as np
from backend_ndarray.ndarray import *

np.random.seed(0)
device = cuda()
# Dimensions of the matrix
rows, cols = 10, 10
nonzero_elements = 10

# Initialize a sparse matrix with all zeros
matrix = np.zeros((rows, cols))

# Randomly generate indices for nonzero elements
row_indices = np.random.choice(rows, nonzero_elements, replace=True)
col_indices = np.random.choice(cols, nonzero_elements, replace=True)

# Generate random values for the nonzero elements
values = np.random.random(nonzero_elements)

# Populate the matrix
for r, c, v in zip(row_indices, col_indices, values):
    matrix[r, c] = v

orig_matrix = NDArray(matrix, device=device)
orig_matrix

/content/drive/MyDrive/10714/final_proj/gnn_with_spmm/python/needle


NDArray([[0.         0.         0.         0.         0.         0.
  0.07103606 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.7991586  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.87001216 0.0202184  0.        ]
 [0.         0.46147937 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.9786183  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.83261985 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.   

#### Then, we are going to transform it to a sparse matrix.

In [ ]:
sparse_matrix = orig_matrix.to_sparse()
sparse_matrix

SparseMatrix(nnz=8, shape=(10, 10),
  data=[0.07103606 0.7991586  0.87001216 0.0202184  0.46147937 0.9786183
 0.83261985 0.77815676],
  row_indices=[0 2 3 3 4 5 7 9],
  col_indices=[6 8 7 8 1 7 1 6])

As you can see, the sparse matrix contains three length 10 array, `data`, `row_indices` and `col_indices`. The `data` represents the values of the all the non-zero elements inside the matrix, while `row_indices` and `col_indices` represents the row and column index of each non-zero element's index.

We can also switch the sparse matrix back to dense matrix by calling `to_dense()` function.



In [ ]:
dense_matrix = sparse_matrix.to_dense()
dense_matrix

NDArray([[0.         0.         0.         0.         0.         0.
  0.07103606 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.7991586  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.87001216 0.0202184  0.        ]
 [0.         0.46147937 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.9786183  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.83261985 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.   

## 4. Sparse Matrix Multiplication
### COO Format Matrix Multiplication

Matrix multiplication in the **COO (Coordinate)** format involves multiplying two sparse matrices that are represented in the COO format. Since the COO format only stores nonzero elements along with their row and column indices, performing multiplication requires processing each nonzero element and its corresponding coordinates.

#### Step 1: Represent the Matrices in COO Format
Assume two matrices A and B, both stored in COO format. They are represented by the following components:

- **Matrix A**:
  - `A_data`: Nonzero values in matrix A.
  - `A_row`: Row indices of the nonzero values in A.
  - `A_col`: Column indices of the nonzero values in A.

- **Matrix B**:
  - `B_data`: Nonzero values in matrix B.
  - `B_row`: Row indices of the nonzero values in B.
  - `B_col`: Column indices of the nonzero values in B.

Let matrix A be of size m × n and matrix B be of size n × p. The resulting matrix C will be of size m × p.

#### Step 2: Initialize the Resultant Matrix
The resulting matrix C will also be sparse and initially contain only zeros. In COO format, the result will have:

- `C_data`: Nonzero values of the resulting matrix.
- `C_row`: Row indices of the nonzero values in C.
- `C_col`: Column indices of the nonzero values in C.

#### Step 3: Compute Nonzero Elements of the Result
To calculate C = A × B, follow these steps for each nonzero element of A:

1. **Find the corresponding element in matrix B**: For each nonzero element A_ij in A, find the corresponding column indices of B. The row index in B must match the column index in A to compute the dot product.

2. **Perform the multiplication**: For each pair of nonzero elements A_ij and B_jk, multiply them together and add to the corresponding entry in C:
   
   ```
   C_ik = C_ik + A_ij × B_jk
   ```

3. **Store the result**: If C_ik is nonzero after the above addition, store the result in the COO format:
   - Add the value of C_ik to `C_data`.
   - Add the row index i to `C_row`.
   - Add the column index k to `C_col`.

#### Step 4: Handle Sparse Properties
Since the result matrix C is also sparse, ensure that only nonzero values are stored. If the sum of C_ik is zero, it should not be stored in the COO format.
#### Define two sparse matrices.


In [ ]:
m, n, p = 500, 500, 500
device = cuda()
nnz = 100
# Initialize a sparse matrix with all zeros
matrix1 = np.zeros((m, n))
matrix2 = np.zeros((n, p))

# Randomly generate indices for nonzero elements
row_indices_1 = np.random.choice(m, nnz, replace=True)
col_indices_1 = np.random.choice(n, nnz, replace=True)
row_indices_2 = np.random.choice(n, nnz, replace=True)
col_indices_2 = np.random.choice(p, nnz, replace=True)

# Generate random values for the nonzero elements
values_1 = np.random.random(nnz)
values_2 = np.random.random(nnz)

# Populate the matrix
for r, c, v in zip(row_indices_1, col_indices_2, values_1):
    matrix1[r, c] = v
for r, c, v in zip(row_indices_2, col_indices_2, values_2):
    matrix2[r, c] = v

dense_matrix1 = NDArray(matrix1, device=device)
dense_matrix2 = NDArray(matrix2, device=device)
dense_matrix1, dense_matrix2

(NDArray([[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]], device=cuda()),
 NDArray([[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]], device=cuda()))

In [ ]:
# Convert dense matrices to sparse matrices.
sparse_matrix1 = dense_matrix1.to_sparse()
sparse_matrix2 = dense_matrix2.to_sparse()
sparse_matrix1, sparse_matrix2

(SparseMatrix(nnz=100, shape=(500, 500),
   data=[0.10029394 0.46357542 0.05537432 0.76532525 0.37416998 0.35561273
  0.6304479  0.29214752 0.4783703  0.40724117 0.3960597  0.58641016
  0.13248764 0.96157014 0.6178767  0.23170163 0.79920256 0.07952208
  0.96193635 0.77058077 0.1314828  0.29302028 0.14484777 0.49739137
  0.97749513 0.18327984 0.42468548 0.9473706  0.63947254 0.21550767
  0.09784448 0.22431703 0.8638556  0.72559434 0.11753186 0.05342718
  0.20747007 0.24536721 0.7168597  0.82211775 0.14694664 0.01323686
  0.27762872 0.9065555  0.8621915  0.58678436 0.9037197  0.9295293
  0.08960304 0.14814086 0.02566272 0.87650526 0.08342244 0.9608347
  0.21331197 0.5654213  0.42053947 0.06395527 0.8605512  0.23223414
  0.66991657 0.3472335  0.511319   0.55219245 0.08110139 0.7270443
  0.4856276  0.26211816 0.13690028 0.13206811 0.6720478  0.33314514
  0.01642963 0.2703279  0.58447605 0.9493188  0.94043195 0.87428796
  0.7308558  0.7740473  0.68328136 0.9413777  0.5573688  0.48805627
  0

#### Matrix Multiplication between dense matrices.

In [ ]:
import time
start = time.time()
correct_result = dense_matrix1 @ dense_matrix2
end = time.time()
print(f"Time taken for dense-dense matrix multipliction: {(end - start)*1000} ms")
correct_result

Time taken for dense-dense matrix multipliction: 0.3287792205810547 ms


NDArray([[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], device=cuda())

#### Matrix Multiplication between sparse matrices.

In [ ]:
start = time.time()
result = sparse_matrix1 @ sparse_matrix2
print(result)
# result = result.to_dense() # when flag = True return dense
end = time.time()
print(f"Time taken for sparse-sparse matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Time taken for sparse-sparse matrix multipliction: 3.0074119567871094 ms
Result Correction: 
True


In [ ]:
start = time.time()
result = sparse_matrix1 @ dense_matrix2
result = result
end = time.time()
print(f"Time taken for sparse-dense matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

Time taken for sparse-dense matrix multipliction: 0.49948692321777344 ms
Result Correction: 
True


In [ ]:
start = time.time()
result = dense_matrix1 @ sparse_matrix2
result = result
end = time.time()
print(f"Time taken for dense-sparse matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

Time taken for dense-sparse matrix multipliction: 0.3407001495361328 ms
Result Correction: 
True


In [ ]:
!python -m backend_ndarray.ndarray

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'backend_ndarray.ndarray' found in sys.modules after import of package 'backend_ndarray', but prior to execution of 'backend_ndarray.ndarray'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Time for dense @ dense: 0.225067138671875 ms
Time for dense @ sparse: 0.9086132049560547 ms
Time for sparse @ dense: 0.2598762512207031 ms
Time for sparse @ sparse: 0.09012222290039062 ms
Total time: 1.2586116790771484 ms
dense @ sparse: True
sparse @ dense: True
sparse @ sparse: True


## Cora Dataset

The Cora dataset contains 2,708 scientific publications, each classified into one of seven classes. Each publication is represented by a binary word vector, indicating the presence or absence of the corresponding word from a dictionary of 1,433 unique words. This dataset is widely used in graph-based machine learning tasks such as node classification and link prediction.

*   Nodes: Publications (e.g. papers, books)
*   Edges: Citation relationships
*   Node Features: 0/1
*   Labels: 7 publication types (Reinforcement Learning, Case Based, Rule Learning, Genetic Algorithms, Probabilistic Methods, Neural_Networks, Theory)

Cora dataset includes two files:
*   ./data/cora/cora.content: The first element represents the node name and the final element represents the label of the node. The left elements represent the node features.

*   ./data/cora/cora.cites: Each line indicates the tuple of connected nodes.

In [14]:
!git pull

X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {'Reinforcement_Learning', 'Case_Based', 'Rule_Learning', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Neural_Networks', 'Theory'}

Number of classes:  7


In [15]:
import os
import random
import numpy as np
import scipy.sparse as sp

# Paths to raw data
content_file = './data/cora/cora.content'
cites_file = './data/cora/cora.cites'

# Load the raw data
all_data = []
with open(content_file, 'r') as f:
    all_data = f.read().splitlines()

all_edges = []
with open(cites_file, 'r') as f:
    all_edges = f.read().splitlines()

# Shuffle the data (nodes are originally grouped by label)
random_state = 77
random.seed(random_state)
random.shuffle(all_data)

# Parse nodes, features, labels
labels = []
nodes = []
X = []
for data in all_data:
    elements = data.split('\t')
    node_id = elements[0]
    feature_vec = elements[1:-1]   # binary feature presence
    label = elements[-1]
    nodes.append(node_id)
    X.append(feature_vec)
    labels.append(label)

X = np.array(X, dtype=int)
N = X.shape[0]  # number of nodes
F = X.shape[1]  # number of features

print('X shape:', X.shape)

# Build a mapping from node_id to index
node_to_idx = {node: i for i, node in enumerate(nodes)}

# Parse edges and build adjacency matrix
edge_list = []
for edge in all_edges:
    e = edge.strip().split('\t')
    src, dst = e[0], e[1]
    if src in node_to_idx and dst in node_to_idx:
        edge_list.append((node_to_idx[src], node_to_idx[dst]))
        edge_list.append((node_to_idx[dst], node_to_idx[src]))  # undirected graph

# Create adjacency matrix in sparse format
row, col = zip(*edge_list)
data = np.ones(len(row))
A = sp.csr_matrix((data, (row, col)), shape=(N, N))

print('\nNumber of nodes (N):', N)
print('Number of features (F):', F)

unique_labels = list(set(labels))
print('\nCategories:', unique_labels)
num_classes = len(unique_labels)
print('Number of classes:', num_classes)

# Map labels to integers
label_to_idx = {label: i for i, label in enumerate(unique_labels)}
y = np.array([label_to_idx[l] for l in labels], dtype=int)

# Print some stats
print("\nData statistics:")
print("Number of nodes:", N)
print("Number of features:", F)
print("Number of classes:", num_classes)
print("Adjacency matrix shape:", A.shape)
print("Feature matrix shape:", X.shape)
print("Label vector shape:", y.shape)

# save
np.save("data/cora_features.npy", X)
np.save("data/cora_labels.npy", y)
sp.save_npz("data/cora_adj.npz", A)

# After saving these files, you can use the previously discussed CoraDataset and GCN model training pipeline.

X shape: (2708, 1433)

Number of nodes (N): 2708
Number of features (F): 1433

Categories: ['Reinforcement_Learning', 'Case_Based', 'Rule_Learning', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Neural_Networks', 'Theory']
Number of classes: 7

Data statistics:
Number of nodes: 2708
Number of features: 1433
Number of classes: 7
Adjacency matrix shape: (2708, 2708)
Feature matrix shape: (2708, 1433)
Label vector shape: (2708,)


In [ ]:
import sys
sys.path.append('./python')
sys.path.append('./apps')

import needle as ndl
import numpy as np

from needle.nn_basic import SoftmaxLoss
from needle.optim import Adam

device = ndl.cpu()  # or ndl.cuda() if you have GPU support

# Paths to preprocessed data
feature_path = "data/cora_features.npy"
adj_path = "data/cora_adj.npz"
label_path = "data/cora_labels.npy"

# Initialize the dataset
dataset = ndl.data.CoraDataset(feature_path, adj_path, label_path, device=device)
X, A, y, idx_train = dataset.get_train_data()
_, _, _, idx_val = dataset.get_val_data()
_, _, _, idx_test = dataset.get_test_data()

# Extract shapes and number of classes
in_features = X.shape[1]
num_classes = len(np.unique(y.numpy()))
hidden_features = 16

# Initialize the GCN model
model = ndl.nn.GCN(in_features, hidden_features, num_classes, device=device)

# Define optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = SoftmaxLoss()

n_epochs = 200
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(X, A)

    # Compute training loss only on training nodes
    train_out = out[idx_train, :]
    train_labels = y[idx_train]
    loss = loss_fn(train_out, train_labels)

    # Backward pass
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.numpy():.4f}")

# Evaluation
model.eval()
out = model(X, A)

# Evaluate on test set
test_out = out[idx_test, :]
test_labels = y[idx_test].numpy()
preds = test_out.numpy().argmax(axis=1)
acc = (preds == test_labels).mean()
print(f"Test Accuracy: {acc:.4f}")

## Graph Neural Network(GNN)
A Graph Neural Network (GNN) is a type of neural network specifically designed to process and analyze data structured as graphs. In a graph, data is represented as nodes (vertices) connected by edges (links), which may have associated features or attributes.
### Common GNN Architectures:
1. Graph Convolutional Networks (GCN): Generalizes the concept of convolution to graphs.
2. Graph Attention Networks (GAT): Uses attention mechanisms to weigh neighbor contributions.
3. GraphSAGE: Focuses on efficient sampling and aggregating information from neighbors.

In this project, we are going to implement Graph Convolutional Network(GCN) and use sparse matrix multiplication mechnisms we implemented to accelerate its forward and backward passes.